In [132]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import uuid
import glob


In [133]:
# make sure you have "data" directory in parallel to "notebooks". Create "input" directory under
# "data" directory and copy sample image file.
# application creates "output" programmatically.
# ----+/notebooks
# ----+/data
# ----------+/input

base_dir                   = '/Users/kd/Workspace/python/github/handwriting-recognition'
data_dir                   = 'data'
input_data_dir             = 'input'
output_data_dir            = 'output'

output_cropped_data_dir        = 'cropped'
output_extracted_data_dir      = 'extracted'
output_letter_cropped_data_dir = 'letter-cropped'
input_filename                 = 'sample_input_02.jpg'

In [134]:
# utility function
def create_directory(path):
    try:
        os.mkdir(path)
        return True
    except FileExistsError as fe_error:
        return True
    except OSError as error:
        print(error)
    return False

# read file directory
def read_directory(path, pattern='*'):
    files = [f for f in glob.glob(os.path.join(path, pattern))]
    return files

def show_img(img):
    plt.axis('off')
    plt.figure(figsize=(10,10))
    plt.imshow(img);

In [135]:
# identify number of tables present on the given image file
def get_table_bounding_boxes(threshold, image):
    kernel          = np.ones((3,3), 'uint8')
    par_img         = cv2.dilate(threshold, kernel, iterations=2)
    _, contours, _  = cv2.findContours(par_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# save table images separately
def get_relevant_images(img, contours, allowed_area=50000):
    images = []
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if (w*h) > allowed_area:
            crop_img = img[y:y+h, x:x+w]
            images.append(crop_img)
    return images


def extract_save_tables(filepath, extraction_path):
    image           = cv2.imread(filepath)
    gray            = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, threshold  = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    contours        = get_table_bounding_boxes(threshold, image)
    images          = get_relevant_images(image, contours)
    print("found (%d) image of interest in the given filepath (%s)"%(len(images), filepath))
    for index, image in enumerate(images):
        temp_filepath = os.path.join(extraction_path, "extract_" + str(index) + ".jpg")
        print("saving table at (%s)"%(temp_filepath))
        cv2.imwrite(temp_filepath, image)


In [166]:
# detect horizontal and vertical lines in the cropped images and extract boxes
def smoothen_out_image(image):
    edges  = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 3, -2)
    kernel = np.ones((3, 3), np.uint8)
    edges  = cv2.dilate(edges, kernel)
    smooth = np.copy(image)
    smooth = cv2.blur(smooth, (2, 2))
    (rows, cols) = np.where(edges != 0)
    image[rows, cols] = smooth[rows, cols]
    return image

def combine_image(vertical_img, horizontal_img):
    alpha  = 0.5
    beta   = 1.0 - alpha
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
    img_final = cv2.addWeighted(vertical_img, alpha, horizontal_img, beta, 0.0)
    img_final = cv2.erode(~img_final, kernel, iterations=2)
    (thresh, img_final) = cv2.threshold(img_final, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return img_final

def extract_boxes_from_image(filepath):
    src_img      = cv2.imread(filepath, cv2.IMREAD_COLOR)
    gray_img     = src_img
    if len(src_img.shape) == 3:
        gray_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY)

    gray_img = cv2.bitwise_not(gray_img)
    bw_img   = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -2)
    
    horizontal_img = np.copy(bw_img)
    vertical_img   = np.copy(bw_img)
    
    cols = horizontal_img.shape[1]
    horizontal_size = cols // 30
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
    horizontal_img = cv2.erode(horizontal_img, horizontalStructure)
    horizontal_img = cv2.dilate(horizontal_img, horizontalStructure)
    
    rows = vertical_img.shape[0]
    vertical_size = rows // 30
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, vertical_size))
    vertical_img = cv2.erode(vertical_img, verticalStructure)
    vertical_img = cv2.dilate(vertical_img, verticalStructure)
    
    horizontal_img = smoothen_out_image(horizontal_img)
    vertical_img   = smoothen_out_image(vertical_img)
    
    img_final      = combine_image(vertical_img, horizontal_img)
    return src_img, img_final

def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)


# resize the image by joining the image
def resize_image(img, size=(64,64)):
    h, w = img.shape[:2]
    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)
    dif = h if h > w else w
    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC
    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

# let's extract characters from the detected table boxes
def extract_individual_characters(filepath, output_dir):
#     output_dir = os.path.join(os.path.dirname(filepath), 'cropped')
#     ret = create_directory(output_dir)
    
#     file_output_cropped_dir = os.path.join(output_dir, os.path.splitext(os.path.basename(filepath))[0])
#     ret = create_directory(file_output_cropped_dir)
    
    gray_img = cv2.imread(filepath, 0)
    # gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    gray_img = cv2.bitwise_not(gray_img)
    #bw_img   = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -2)
    bw_img   = cv2.adaptiveThreshold(gray_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,15,-2)
    # find contours and get the external one
    _, contours, hier = cv2.findContours(bw_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort all the contours by top to bottom.
    (contours, boundingBoxes) = sort_contours(contours, method='left-to-right')
    
    print("processing contours for filename: %s"%(filepath))
    
    cont_ind = 0
    for index, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        if h > 5 and w < 50:
            filename = os.path.join(output_dir, str(int(cont_ind/len(contours)))+"_"+str(int(cont_ind%len(contours)))+"_"+os.path.basename(filepath))
            print(filename)
            crop_img = gray_img[y:y+h, x:x+w]
            crop_img = resize_image(crop_img)
            cv2.imwrite(filename, crop_img)
            cont_ind = cont_ind + 1

In [167]:
# program initialization 
img_filename    = os.path.join(base_dir, data_dir, input_data_dir, input_filename)
print("input filename : [%s]"%(img_filename))

output_basedir  = os.path.join(base_dir, data_dir, output_data_dir, os.path.splitext(input_filename)[0])
print("output working directory: [%s]"%(output_basedir))
ret = create_directory(os.path.join(base_dir, data_dir, output_data_dir))
ret = create_directory(output_basedir)

output_cropped_dir = os.path.join(output_basedir, output_cropped_data_dir)
print("output cropped directory: [%s]"%(output_cropped_dir))
ret = create_directory(output_cropped_dir)

output_extracted_dir = os.path.join(output_basedir, output_extracted_data_dir)
print("output extracted directory: [%s]"%(output_extracted_dir))
ret = create_directory(output_extracted_dir)

output_letter_cropped_dir = os.path.join(output_basedir, output_letter_cropped_data_dir)
print("output cropped directory: [%s]"%(output_letter_cropped_dir))
ret = create_directory(output_letter_cropped_dir)



input filename : [/Users/kd/Workspace/python/github/handwriting-recognition/data/input/sample_input_02.jpg]
output working directory: [/Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02]
output cropped directory: [/Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/cropped]
output extracted directory: [/Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/extracted]
output cropped directory: [/Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/letter-cropped]


In [168]:
# let's extract all the tables present and save them individually
image, image_processed = extract_boxes_from_image(img_filename)
_, contours, hierarchy = cv2.findContours(image_processed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
(contours, boundingBoxes) = sort_contours(contours, method='top-to-bottom')
print("extracting tables for filename: [%s]" % (img_filename))

for index, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    if (w*h > 300000) and (w*h < 800000):
        filename = os.path.join(output_extracted_dir, "table_" + str(index)+ "_" + os.path.basename(img_filename))
        crop_img = image[y:y+h, x:x+w]
        cv2.imwrite(filename, crop_img)

extracting tables for filename: [/Users/kd/Workspace/python/github/handwriting-recognition/data/input/sample_input_02.jpg]


In [169]:
extracted_files = read_directory(output_extracted_dir)

for filepath in extracted_files:
    file_output_cropped_dir = output_cropped_dir
    ret = create_directory(file_output_cropped_dir)

    image, image_processed = extract_boxes_from_image(filepath)
    # Find contours for image, which will detect all the boxes
    _, contours, hierarchy = cv2.findContours(image_processed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort all the contours by top to bottom.
    (contours, boundingBoxes) = sort_contours(contours, method='top-to-bottom')
    print("processing contours for filename: %s"%(filepath))
    cont_ind = 0
    for index, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        if (w > 80 and h > 20) and (w*h < 100000):
            filename = os.path.join(file_output_cropped_dir, str(int(cont_ind/5))+"_"+str(int(cont_ind%5))+"_"+os.path.basename(filepath))
            crop_img = image[y:y+h, x:x+w]
            cv2.imwrite(filename, crop_img)
            cont_ind = cont_ind + 1

processing contours for filename: /Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/extracted/table_1_sample_input_02.jpg
processing contours for filename: /Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/extracted/table_53_sample_input_02.jpg


In [170]:
# path1 = '/Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/cropped/sample_input_02/13_0_3_table_53_sample_input_02.jpg'
# extract_individual_characters(path1)

In [172]:

extracted_cropped_files = read_directory(output_cropped_dir)
for filepath in extracted_cropped_files:
    file_output_cropped_dir = os.path.join(output_letter_cropped_dir, os.path.splitext(os.path.basename(filepath))[0])
    ret = create_directory(file_output_cropped_dir)
    extract_individual_characters(filepath, file_output_cropped_dir)

processing contours for filename: /Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/cropped/6_0_table_1_sample_input_02.jpg
processing contours for filename: /Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/cropped/7_1_table_1_sample_input_02.jpg
/Users/kd/Workspace/python/github/handwriting-recognition/data/output/sample_input_02/letter-cropped/7_1_table_1_sample_input_02/0_0_7_1_table_1_sample_input_02.jpg
